# Compare W2V models results over eval.set
## TODO: Weighted mean of vectors

In [1]:
# load eval.set (duplicates free)
import json
filename = 'pre-clustered_docs_harder.json'
#filename = 'english_3_clusters.json' we need an english wikipedia model for this
with open(filename, 'r') as file:
    cdocs = json.load(file)
#import sys
#!{sys.executable} -m pip install plotly


# Load entities models
## TODO: Check entities format, does it really match the word in the vocabulary?

In [2]:
# load pre-trained model
from gensim.models import Word2Vec

unwrapped_docs = [doc for cluster in cdocs for doc in cluster]

wiki_model = Word2Vec.load('./models/wiki_iter=5_algorithm=skipgram_window=10_size=300_neg-samples=10.m')
#my_model = Word2Vec.load('./models/w2v_entities+abstract_model.model')
print("Models loaded")

C:\Users\nick_\Anaconda3\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


Models loaded


C:\Users\nick_\Anaconda3\lib\site-packages\gensim\models\base_any2vec.py:743: UserWarning: C extension not loaded, training will be slow. Install a C compiler and reinstall gensim for fast training.
  "C extension not loaded, training will be slow. "


In [4]:
import numpy as np  # Make sure that numpy is imported

def makeFeatureVec(words, model, num_features):
    # Function to average all of the word vectors in a given
    # paragraph
    #
    # Pre-initialize an empty numpy array (for speed)
    featureVec = np.zeros((num_features,),dtype="float32")
    #
    nwords = 0.
    # 
    # Index2word is a list that contains the names of the words in 
    # the model's vocabulary. Convert it to a set, for speed 
    index2word_set = set(model.wv.index2word)
    #
    # Loop over each word in the review and, if it is in the model's
    # vocaublary, add its feature vector to the total
    for word in words:
        if word in index2word_set: 
            nwords = nwords + 1.
            featureVec = np.add(featureVec, model[word])
        elif word.lower() in index2word_set:
            nwords += 1.
            featureVec = np.add(featureVec, model[word.lower()])
    # 
    # Divide the result by the number of words to get the average
    featureVec = np.divide(featureVec,nwords)
    return featureVec


def getAvgFeatureVecs(docs, model, num_features):
    # Given a set of reviews (each one a list of words), calculate 
    # the average feature vector for each one and return a 2D numpy array 
    # 
    # Initialize a counter
    counter = 0
    # 
    # Preallocate a 2D numpy array, for speed
    reviewFeatureVecs = np.zeros((len(docs),num_features),dtype="float32")
    # 
    # Loop through the reviews
    for doc in docs:
       #
       # Print a status message every 1000th review
        #if counter%1000. == 0.:
        #    print("Review %d of %d" % (counter, len(docs)))
       # 
       # Call the function (defined above) that makes average feature vectors
        reviewFeatureVecs[counter] = makeFeatureVec(doc['result_entities'], model, \
           num_features)
       #
       # Increment the counter
        counter = counter + 1
    return reviewFeatureVecs
print(my_model['re'].shape)
trainDataVecs = getAvgFeatureVecs( unwrapped_docs, my_model, my_model.wv.vector_size )
print(trainDataVecs)
#urls_cluster_list = my_dbscan.perform_dbscan(eps = 0.4, min_samples = 4, data = trainDataVecs, verbose = True
#                  , titles = None, urls = None, print_noise = True)
db, clusters = my_dbscan.perform_dbscan(eps = 0.27, min_samples = 2, data = my_doc_vecs, verbose = True
                  , titles = titles, urls = urls, print_noise = True)
print(clusters, db)

(100,)
[[-0.4029415  -0.13100496  0.11882563 ... -0.33560228 -0.11999029
   0.24970573]
 [-0.33952245 -0.22362994  0.06498575 ... -0.28428683 -0.10469311
   0.16570613]
 [-0.3010722  -0.08241285  0.12940371 ... -0.35231194 -0.03992369
   0.22636156]
 ...
 [-0.45489705  0.24162287  0.19298626 ... -0.348121   -0.03367177
  -0.03407436]
 [-0.72975105  0.35789543  0.35934475 ... -0.50478923 -0.03901616
  -0.08851031]
 [-0.723315    0.32045954  0.33745176 ... -0.49936923 -0.03127839
  -0.07138227]]
{0: []} DBSCAN(algorithm='auto', eps=0.27, leaf_size=30, metric='euclidean',
    metric_params=None, min_samples=2, n_jobs=None, p=None)


/home/nick/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:54: DeprecationWarning:

Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).

/home/nick/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:21: DeprecationWarning:

Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).

/home/nick/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:24: DeprecationWarning:

Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).



In [14]:
# check unknown words to model
import numpy as np
import w2v_model_utils as mm_utils
import importlib
importlib.reload(mm_utils)
model = wiki_model
unknown = []
n_words = 0
index2word_set = set(model.wv.index2word)
for doc in unwrapped_docs:
    mm_utils.infer_vector(doc['result_entities'], model, index2word_set, verbose=True) 
                    
#print(unknown, len(unknown),'over', n_words)
#model['elon musk']
model.most_similar(positive=['re', 'donna'], negative=['uomo'])
#model['energia']
print(len([e for d in unwrapped_docs for e in d['result_entities']]))

Known words: 96.66666666666667%  (29/30)
Unknown entities found:  ['Radioattivita']
Known words: 100.0%  (9/9)
Known words: 100.0%  (23/23)
Known words: 100.0%  (17/17)
Known words: 75.0%  (3/4)
Unknown entities found:  ['Tinder']
Known words: 92.85714285714286%  (13/14)
Unknown entities found:  ['Tinder']
Known words: 92.3076923076923%  (12/13)
Unknown entities found:  ['Tinder']
Known words: 100.0%  (15/15)
Known words: 100.0%  (18/18)
Known words: 100.0%  (26/26)
Known words: 100.0%  (28/28)
Known words: 97.72727272727273%  (43/44)
Unknown entities found:  ['Perpendicolarita']
Known words: 100.0%  (7/7)
Known words: 100.0%  (21/21)
Known words: 96.92307692307692%  (63/65)
Unknown entities found:  ['Pro12', 'CarPlay']
Known words: 100.0%  (20/20)
Known words: 100.0%  (61/61)
Known words: 100.0%  (13/13)
Known words: 100.0%  (18/18)
Known words: 97.87234042553192%  (92/94)
Unknown entities found:  ['1080p', 'IBeacon']
Known words: 93.33333333333333%  (14/15)
Unknown entities found:  [

C:\Users\nick_\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  from ipykernel import kernelapp as app
C:\Users\nick_\Anaconda3\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [10]:
#model['Elon Musk'] this one is known elon musk is not; consider capitalizing first letters
import unicodedata
output = unicodedata.normalize('NFD', u"radioattività").encode('ascii', 'ignore')

import unicodedata
import re


def strip_accents(text):
    """
    Strip accents from input String.

    :param text: The input string.
    :type text: String.

    :returns: The processed String.
    :rtype: String.
    """
    try:
        text = unicode(text, 'utf-8')
    except (TypeError, NameError): # unicode is a default on python 3 
        pass
    text = unicodedata.normalize('NFD', text)
    text = text.encode('ascii', 'ignore')
    text = text.decode("utf-8")
    return str(text)

def text_to_id(text):
    """
    Convert input text to id.

    :param text: The input string.
    :type text: String.

    :returns: The processed String.
    :rtype: String.
    """
    text = strip_accents(text)
    #text = re.sub('[ ]+', '_', text)
    #text = re.sub('[^0-9a-zA-Z_-]', '', text)
    return text
wiki_model[text_to_id("Luminosità").lower()]

#text_to_id("Montréal")

C:\Users\nick_\Anaconda3\lib\site-packages\ipykernel_launcher.py:42: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


KeyError: "word 'Sessualita' not in vocabulary"

Observation: I found many of the problems I expected, in fact the model hasn't picked up on composed-words, and it mostly knows lower-cased ones; ex: Marte (astronomia), is the name of the wiki page linked to it, but we don't have that token, nor we have a way to map wikipedia pages to single words (like Marte(astronomia) to marte). 
My solution was to split every word, and create a vector from this by adding them up; if a word is unknown, like (astronomia), the other one is used, and in this case this works pretty well. 
Words containing accents are still a problem

# Evaluate Clustering

In [17]:
import utils
import my_dbscan
import model_evaluation as me
import w2v_model_utils as mm_utils
import numpy as np
import importlib
importlib.reload(mm_utils)
from gensim.utils import simple_preprocess as sp

# infer document vectors (once, they won't change)
# here's a problem we didn't consider: some docs might not have a representation at all 
# (all its entities are unknown to the model); we have to keep track of those.
my_doc_vecs = []
unknown_i = []
index2word_set = set(wiki_model.wv.index2word)
print("Number of docs: ", len(unwrapped_docs))
for i, doc in enumerate(unwrapped_docs):
    dvec = mm_utils.infer_vector(doc['result_entities'], wiki_model, index2word_set, verbose=False)
    if len(dvec) > 0:
        my_doc_vecs.append(dvec)
    else:
        unknown_i.append(i)
#print(my_doc_vecs[:2])
#wiki_doc_vecs = [mm_utils.infer_vector(doc['result_entities'], wiki_model) for doc in unwrapped_docs]
# get titles and urls, ONLY from valid docs
titles = [doc['title'] for j, doc in enumerate(unwrapped_docs) if not(j in unknown_i)]
urls = [doc['url'] for j, doc in enumerate(unwrapped_docs) if not(j in unknown_i)]
assert len(titles) == len(urls)
print("Length of titles retrieved %s, and of docvecs %s" %(len(titles), len(my_doc_vecs)))

models_recall = []
models_precision = []
clusters_found = [] # keep clusters results, they're useful later on
min_s = 2 # min_samples

# let's try 3 combinations of clustering
for k in range(3):
    # try different eps for some model
    if k == 0:
        # default eps dbscan
        eps = 0.27
        eps_increment = 0.1
    elif k==1:
        eps = 0.11
        eps_increment = 0.04
    else:
        eps = 0.094
        eps_increment = 0.04
        """
        # online update of model
        print("#######Online-training########")
        eps = 0.11
        eps_increment = 0.13
        #training_corpus = [(doc['result_entities']) for doc in unwrapped_docs]
        #train on whole data-set
        training_corpus = [ sp(doc['title'], doc['abstract']) for doc in unwrapped_docs]

        # Re-training is really long on this model
        %time wiki_model = mm_utils.online_training(wiki_model, training_corpus, epochs=7)
        # re-infer vectors
        my_doc_vecs2 = [mm_utils.infer_vector(doc['result_entities'], wiki_model, index2word_set, verbose=False)
                       for doc in unwrapped_docs]
        # missing new titles here
        """
    doc_vecs = my_doc_vecs
    #if k==2:
    #    doc_vecs = my_doc_vecs2
    # TODO: change to 2 to cluster both models
    for i in range(1):
        # apply dbscan clustering to these vectors
        urls_cluster_list = my_dbscan.apply_dbscan(doc_vecs = doc_vecs, titles = titles, 
                                                       urls = urls, subset_length = len(doc_vecs),
                                                     eps = eps, eps_increment = eps_increment,
                                                   n_iterations = 3, verbose = False, min_samples = min_s)
        # get clusters as list of titles
        titles_clusters = utils.getDocTitleFromUrl(unwrapped_docs, urls_cluster_list)
        clusters_found.append(titles_clusters)
        #data = utils.plot_clusters(titles_clusters)

        # evaluate clustering
        expected_clusters = []
        for docs in cdocs:
            expected_clusters.append([doc['title'] for doc in docs])
        precision, recall = me.compute_clustering_accuracy(titles_clusters, expected_clusters)
        print('Precision score: %s, Recall score: %s'%(precision, recall))

        models_precision.append(precision)
        models_recall.append(recall)
        # last check: we want to penalize models that simply cluster all docs together (that's not a valid result)
        # that's way we have precision score
        print("#Number of clusters found: {0}, against number of pre-computed clusters: {1}#\n".format(
            len(urls_cluster_list), len(cdocs)))
        # change inferred docs for next model
        #doc_vecs = wiki_doc_vecs

Number of docs:  54
Length of titles retrieved 54, and of docvecs 54
Accuracy (Precision) over each cluster:  [7.4074074074074066, 5.555555555555555, 5.555555555555555, 3.7037037037037033, 18.51851851851852, 16.666666666666664, 5.555555555555555, 7.4074074074074066, 18.51851851851852, 11.11111111111111]
Accuracy (Recall) over each cluster:  [100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0]
Precision score: 10.0, Recall score: 100.0
#Number of clusters found: 1, against number of pre-computed clusters: 10#

Accuracy (Precision) over each cluster:  [83.33333333333333, 51.785714285714285, 100.0, 100.0, 35.714285714285715, 67.85714285714285, 7.142857142857142, 66.66666666666666, 41.07142857142857, 33.92857142857143]
Accuracy (Recall) over each cluster:  [50.0, 49.99999999999999, 66.66666666666666, 100.0, 100.0, 25.925925925925924, 66.66666666666666, 50.0, 50.0, 50.0]
Precision score: 58.75, Recall score: 60.925925925925924
#Number of clusters found: 10, against number 

In [5]:
import plotly.plotly as py
import plotly.tools as tls
import plotly.graph_objs as go
# using my api-key
tls.set_credentials_file(username='D4nt3_', api_key='4O71urldgOueVtcApOdX')

title_dist_tuples = utils.choose_eps(2, my_doc_vecs, titles)
trace = go.Scatter(
    x =[x for (x, y) in title_dist_tuples],  # list of x
    y = [y for (x, y) in title_dist_tuples],
    mode = 'lines',
    name = 'lines'
)

data = [trace]

py.iplot(data, filename = 'eps-wiki-model')


In [20]:
for cluster in clusters_found[1]:
    print(cluster)
#print(unwrapped_docs[29])

['Marte, il sottosuolo può avere ospitato la vita - Spazio & Astronomia - ANSA.it', 'NASA, dalla CO2 al glucosio su Marte']
['Decolla Facebook Dating, il Tinder di Menlo Park: primi test in Colombia - Repubblica.it', "iPhone Xs Max tira 3-4 volte più dell'Xs - Hi-tech - ANSA.it", 'Apple conferma "per errore" iPhone XS, XS Max e XR', 'Aspettando iPhone Xs: il giorno di iOS 12', 'iPhone XS: perché Apple ha nascosto il notch?', 'Apple iPhone Xs e Xs Max: è troppo caro?', 'Apple lancia iPhone Xs e la versione Max: sempre più grandi, gli smartphone sono la nuova Tv - Corriere.it', 'Problemi per iPhone XS e XS Max: «Non si caricano se il cavo è collegato mentre lo schermo è spento» - Corriere.it', 'iPhone XS e XS Max: novità, scheda tecnica e prezzo', 'iPhone XS appiana le rughe, protestano gli utenti', 'iPhone XS: proteste per la carica troppo lenta', 'FIFA 19, ecco i requisiti PC', 'YouTube Kids, lo streaming dei bambini in Italia', 'Social e bambini: YouTube assume nuovi moderatori e Face

# Visualize results graphically

In [30]:
import plotly.plotly as py
import plotly.tools as tls
import plotly.graph_objs as go
# using my api-key
tls.set_credentials_file(username='D4nt3_', api_key='4O71urldgOueVtcApOdX')
graph_name = 'model_eval_entities_results'

model_descr = ["DBSCAN Eps=0.11,incr=.04", "K-means with K=10"]
# plot test-accuracy results, this time input values directly (messed up with data structure naming)
# Precision score: 58.75, Recall score: 60.925925925925924
#Number of clusters found: 10, against number of pre-computed clusters: 10#
# Precision score: 54.871212121212125, Recall score: 67.5
trace0 = go.Bar(
    x = model_descr,
    y = [58.75, 54.871212121212125],
    name='Precision',
    marker=dict(
        color='rgb(49,130,189)'
    ),
    width = [0.2, 0.2]
)

trace2 = go.Bar(
    x = model_descr,
    y = [60.925925925925924, 67.5],
    name='Recall',
    marker=dict(
        color='rgb(155, 244, 66)',
    ),
    width = [0.2, 0.2]
    
)
# precision, recall
data = [trace0, trace2]
layout = go.Layout(
    title = 'Clustering Accuracy Results (set-1)',
    xaxis=dict(
        tickfont=dict(
            size=13,
            color='rgb(107, 107, 107)',
            
        ),
        tickangle = 0
    ),
    yaxis=dict(
        title='Clustering accuracy (%)',
        titlefont=dict(
            size=16,
            color='rgb(107, 107, 107)'
        ),
        tickfont=dict(
            size=14,
            color='rgb(107, 107, 107)'
        )
    ),
   
    barmode='group',
    bargap=0.2,
    bargroupgap=0.02
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename=graph_name)

# K-means

In [7]:
from sklearn.cluster import KMeans
#titles = [doc['headline'] for doc in unwrapped_docs]
#titles = [doc['title'] for doc in unwrapped_docs]
# let's try Kmeans with K between 2 and 20
model = wiki_model
def run_KMeans_K_times(doc_vecs, K):
    sse = []
    kmeans_results = []
    # run k means with k between 1 and K
    for i in range(1, K+1):
        kmeans = KMeans(n_clusters=i, init='k-means++', # smart-way of choosing starting point
               n_init= 50, # run k-means 100 times and get the best result in terms of SSE
                n_jobs = -1 # use every core
        )
        # compute sse for this kmeans result
        kmeans.fit(doc_vecs)
        sse.append(kmeans.inertia_)
        
        # get 'visual' clustering, by saving a list of titles
        clusters = {label: [] for label in kmeans.labels_}
        for label, list_ in clusters.items():
            for i, l in enumerate(kmeans.labels_):
                if label==l:
                    clusters[label].append(titles[i])
        
        kmeans_results.append(clusters)
    return kmeans_results, sse

import plotly.plotly as py
import plotly.tools as tls
import plotly.graph_objs as go
# using my api-key
tls.set_credentials_file(username='D4nt3_', api_key='4O71urldgOueVtcApOdX')
graph_name = 'kmeans_choosing_k_WIKI'
my_doc_vecs = []
unknown_i = []
index2word_set = set(model.wv.index2word)
print("Number of docs: ", len(unwrapped_docs))
for i, doc in enumerate(unwrapped_docs):
    dvec = mm_utils.infer_vector(doc['result_entities'], wiki_model, index2word_set, verbose=False)
    if len(dvec) > 0:
        my_doc_vecs.append(dvec)
    else:
        unknown_i.append(i)
        
# get titles and urls, ONLY from valid docs
titles = [doc['title'] for j, doc in enumerate(unwrapped_docs) if not(j in unknown_i)]
## kmeans
kmeans_results, sse = run_KMeans_K_times(my_doc_vecs, 23)
K = 23 # max_k
# plot results of K and SSE
trace = go.Scatter(
    x = [k for k in range(1, K+1)],
    y = sse,
    mode = 'lines+markers'
)
layout = dict(title = 'Choosing K in Kmeans',
              xaxis = dict(title = 'K value'),
              yaxis = dict(title = 'SSE'),
              )

data = [trace]
fig = dict(data=data, layout=layout)
py.iplot(fig, filename = graph_name)

Number of docs:  54


In [12]:
import utils
import my_dbscan
import model_evaluation as me

clusters = []
models_precision_kmeans = []
models_recall_kmeans = []
expected_clusters = []
for docs in cdocs:
    expected_clusters.append([doc['title'] for doc in docs])
    #expected_clusters.append([doc['headline'] for doc in docs])
        

# K already chosen before, with value 4
cluster_results, sse = run_KMeans_K_times(my_doc_vecs, 10)
#cluster_results, sse = run_KMeans_K_times(model, 10)
cluster_results = cluster_results[-1] # get 4-means results
clist = []
for key, cluster in cluster_results.items():
    clist.append(cluster)
clusters.append(clist)
    
# evaluate clustering
precision, recall = me.compute_clustering_accuracy(clist, expected_clusters)
print('Precision score: %s, Recall score: %s'%(precision, recall))

models_precision_kmeans.append(precision)
models_recall_kmeans.append(recall)
# last check: we want to penalize models that simply cluster all docs together (that's not a valid result)
# that's way we have precision score
print("#Number of clusters found: {0}, against number of pre-computed clusters: {1}#\n".format(
    len(clist), len(cdocs)))

Accuracy (Precision) over each cluster:  [50.0, 100.0, 27.27272727272727, 100.0, 49.31818181818182, 40.909090909090914, 53.03030303030303, 50.0, 45.45454545454546, 32.72727272727273]
Accuracy (Recall) over each cluster:  [100.0, 100.0, 100.0, 100.0, 25.0, 50.0, 33.33333333333333, 100.0, 33.333333333333336, 33.333333333333336]
Precision score: 54.871212121212125, Recall score: 67.5
#Number of clusters found: 10, against number of pre-computed clusters: 10#



In [13]:
print(clist)

[['Marte, il sottosuolo può avere ospitato la vita - Spazio & Astronomia - ANSA.it', 'SpaceX: ecco come saranno le basi umane su Marte', 'NASA, dalla CO2 al glucosio su Marte', 'Elon Musk contro la NASA per terraformare Marte', 'A spasso attorno alla Luna, SpaceX annuncia il primo turista spaziale - Corriere.it', 'SpaceX, i viaggi sulla Luna possono attendere', 'SpaceX porterà un uomo in orbita attorno alla Luna', 'SpaceX manderà Yusaku Maezawa sulla Luna'], ['Decolla Facebook Dating, il Tinder di Menlo Park: primi test in Colombia - Repubblica.it', 'Facebook: Tinder nel mirino, novità di Instagram e Whatsapp - Corriere.it', 'Facebook Dating: ecco come funziona l’anti-Tinder di Zuckerberg - Corriere.it'], ['Tesla sotto indagine per colpa dei tweet di Elon Musk: crollo in Borsa - Corriere.it', 'Elon Musk denuciato per truffa, Tesla crolla in borsa - Wired', 'Tesla, Elon Musk lascia la presidenza', 'Fifa 19 contro Pes 2019: qual è il migliore quest’anno? Ecco la sfida giocata da noi - Co

# Silhoutte score
Silhouette coefficient combines ideas of both cohesion and separation, 
but for individual points, as well as clusters and 
clusterings; the Silhouette Coefficient is calculated using the mean intra-cluster distance (a) and the mean nearest-cluster distance (b) for each sample.
The Silhoutte score is merely the average of each silhoutte coefficient, computed over each sample.
It's a measure of the goodness of clustering, by assuming the fact that a cluster X is defined good if both every sample inside it is close to each other, and far from any other relatively-near cluster Y.
This is not necessarily true for every shape of cluster.

    The best value is 1 and the worst value is -1. Values near 0 indicate overlapping clusters.

In [16]:
from sklearn.metrics import silhouette_score
s_scores = []
for model in models:
    # get clusters for each model
    doc_vecs = [model.docvecs[j] for j in range(len(model.docvecs))]
    titles = [doc['title'] for doc in unwrapped_docs]
    urls = [doc['url'] for doc in unwrapped_docs]
    # apply dbscan clustering to these vectors
    urls_cluster_list = my_dbscan.apply_dbscan(doc_vecs = doc_vecs, titles = titles, 
                                               urls = urls, subset_length = len(titles),
                                                 eps = 0.27, eps_increment = 0.1, n_iterations = 3, verbose = False)
    
    # get cluster labels, mantaining original docs ordering
    labels = []
    for doc in unwrapped_docs:
        noise = True
        for i, url_list in enumerate(urls_cluster_list):
            if(doc['url'] in url_list):
                labels.append(i) # keep cluster id
                noise = False
        if noise:
            labels.append(-1)
    # make sure they have the same size
    assert len(labels) == len(doc_vecs)
    ss = silhouette_score(doc_vecs, labels , metric='cosine')
    s_scores.append(ss)
    print(ss)

0.2057951
0.23791973
0.24932371
0.24040304
0.13758796
0.0041218703
0.12820731
0.228402


# Conclusions: 
A really low silhoutte score helps us identify models that tend to have very few cluster, hence not really recognizing differences between docs. This is fundamental, since test rules used so far prevent us from recognizing these kinds of models.
Since silhoutte score is an average of silhoutte coefficients, smaller changes of values here may mean greater differences.